In [97]:
import numpy as np
import pandas as pd
import itertools
import random

In [98]:
data_file = "ques_data.xlsx"    # Question Datafile name
output_file = "output.xlsx"     # Output file name

In [99]:
def read_data(file):
    data = pd.read_excel(file,sheet_name=None)
    sheet = list(data.keys())
    df = data[sheet[0]].astype({"Year":"string"})
    roll_df = data[sheet[1]]
    return df,roll_df
    

In [110]:
df, roll_df = read_data(data_file)

In [111]:
courses = df.columns.to_list()
courses.pop(0)
years = df['Year'].to_list()
df  # Displaying the Question Data

Year  MME213  MME235  EEE267  ME221
0  2018-19       8       8       8      8
1  2017-18      12       8       8      8
2  2016-17      12       8       8      8
3  2015-16      12       8       8      8
4  2014-15      12       8       8      8

In [114]:
roll_df.T # Displaying Roll Numbers of students

0   1   2   3   4   5   6   7   8   9   ...  50  51  52  53  54  55  \
Roll No.   1   2   3   4   5   6   7   8   9  10  ...  51  52  53  54  55  56   

          56  57  58  59  
Roll No.  57  58  59  60  

[1 rows x 60 columns]

In [116]:
def extract_ques(table,q_lst):
    """
        This function Extracts All the Questions from the "table" dataframe and appends those in the "q_lst"
        
        Parameters:
        ----------
        table   -> A dataframe containing "Year" in the first column and "Course Code" in the second column
        q_list  -> A list of strings having the questions in the formate of "CourseCode-Year-(Ques.No.)"

        Returns:
        -------
        q_list

        Example:
        -------
        For the following inputs:
        table = pandas.DataFrame({'Year':['Y1','Y2'],'Course101':[1,2],'Course102':[2,3]})
        q_list = []
        extract_ques(table,q_list) will return the following list:
        q_list = [  'Course101-Y1-(1)',
                    'Course101-Y2-(1)','Course101-Y2-(2)',
                    'Course102-Y1-(1)','Course102-Y1-(2)',
                    'Course102-Y2-(1)','Course102-Y2-(2)','Course102-Y2-(3)'] 

        !!! 
        Note
        Year is a string list
        Course101, Course102 are integer list
        !!!
    """
    crs_code = table.columns.to_list()[1]
    r = len(table.index)                            # Course Code
    for i in range(r): 
        yr = table.iat[i,0]                                          # Each Year
        qs_n = table.iat[i,1]                                        # No. of Questions
        q_no = [str(j) for j in range(1,qs_n+1)]                     # Creates a string list of question no.
        lst_per = list(itertools.product(*[[crs_code],[yr],q_no]))   # List of all question in 'yr' year in 'crs_code' Course as tuples
        for k in lst_per:
            a,b,c = k                                                
            q_lst.append(a+'-'+b+'-'+'('+c+')') 
    return q_lst

def random_dist(st_df,q_list):
    """
        This Function Distributes each questions of "q_list" to dataframe "st_df"
        * It creates new columns named Task1, Task2, ...
        * In each column it distrubutes questions randomly.
        * If no. of questions are less than the row no.s (total student no.) 
          it randomly chooses students and distribute questions among them.
          The remaing students cell is left with a '-'
        * Each time this function is called the "st_df" will change randomly

        Parameters:
        ----------
        st_df   -> Dataframe containing students roll no.
        q_list  -> List of all questions generated from extract_ques() function
        
    """
    n = st_df.index.size
    ts = 1  # Task No.                                    
    while len(q_list):
        if len(q_list)<n:
            remain = n-len(q_list)
            for _ in range(remain):
                q_list.append('-')    
        sample = random.sample(q_list,n)
        for _ in range(n):
            random.shuffle(sample)
        st_df['Task'+str(ts)] = pd.Series(sample)
        ts = ts+1
        for q in sample:
            q_list.remove(q)
    return None

In [117]:
ques_list = []
for c in courses:
    extract_ques(df[['Year',c]],ques_list) 

In [118]:
random_dist(roll_df,ques_list)
roll_df.to_excel(output_file,index=False)    # Saves the "dist" dataframe to output file
roll_df

Roll No.                Task1                Task2                Task3
0          1   MME235-2016-17-(2)   MME235-2014-15-(6)   EEE267-2014-15-(1)
1          2    ME221-2018-19-(1)    ME221-2014-15-(4)   EEE267-2017-18-(4)
2          3   MME213-2015-16-(8)   MME213-2015-16-(4)    ME221-2015-16-(8)
3          4   EEE267-2016-17-(3)   EEE267-2015-16-(8)   EEE267-2017-18-(8)
4          5  MME213-2014-15-(12)   MME213-2018-19-(4)    ME221-2015-16-(5)
5          6   EEE267-2018-19-(2)   EEE267-2018-19-(8)   MME213-2016-17-(6)
6          7    ME221-2016-17-(3)   MME213-2015-16-(9)   MME213-2016-17-(9)
7          8    ME221-2014-15-(7)   EEE267-2015-16-(5)   MME235-2017-18-(3)
8          9   MME213-2017-18-(4)   EEE267-2017-18-(1)    ME221-2015-16-(2)
9         10   MME235-2017-18-(6)   MME213-2018-19-(7)   MME213-2014-15-(5)
10        11   MME235-2018-19-(6)   MME235-2015-16-(1)   MME213-2014-15-(8)
11        12    ME221-2017-18-(3)   MME213-2016-17-(3)   EEE267-2018-19-(1)
12        13    ME221-2017-18-(8)    ME221-2016-17-(1)   MME213-2015-16-(5)
13        14    ME221-2017-18-(4)    ME221-2015-16-(7)   EEE267-2014-15-(3)
14        15   MME235-2015-16-(2)   MME213-2018-19-(8)                    -
15        16   EEE267-2014-15-(7)   MME213-2017-18-(8)                    -
16        17  MME213-2015-16-(11)   MME213-2018-19-(6)    ME221-2018-19-(2)
17        18   EEE267-2014-15-(6)   MME213-2016-17-(1)   EEE267-2016-17-(4)
18        19   MME235-2014-15-(5)   MME213-2016-17-(5)   MME235-2017-18-(8)
19        20   MME213-2014-15-(3)   EEE267-2015-16-(2)   MME235-2017-18-(4)
20        21   MME213-2015-16-(1)    ME221-2018-19-(3)                    -
21        22   MME235-2016-17-(3)   MME213-2014-15-(7)    ME221-2018-19-(8)
22        23   EEE267-2016-17-(7)   EEE267-2016-17-(5)    ME221-2017-18-(6)
23        24    ME221-2018-19-(4)    ME221-2015-16-(6)  MME213-2016-17-(12)
24        25   EEE267-2015-16-(6)   EEE267-2014-15-(8)   MME235-2014-15-(3)
25        26   EEE267-2017-18-(7)  MME213-2016-17-(10)   MME235-2018-19-(2)
26        27    ME221-2014-15-(1)  MME213-2015-16-(12)   EEE267-2017-18-(2)
27        28   MME213-2016-17-(8)   EEE267-2018-19-(3)    ME221-2014-15-(3)
28        29   MME235-2016-17-(8)   EEE267-2018-19-(4)                    -
29        30   MME213-2018-19-(2)   MME213-2016-17-(7)   EEE267-2015-16-(3)
30        31    ME221-2014-15-(6)   EEE267-2014-15-(4)   MME235-2018-19-(4)
31        32   EEE267-2017-18-(3)   MME213-2018-19-(1)   MME235-2017-18-(5)
32        33   MME235-2014-15-(4)    ME221-2017-18-(5)   MME235-2014-15-(7)
33        34   MME235-2015-16-(6)   MME213-2017-18-(5)  MME213-2016-17-(11)
34        35    ME221-2017-18-(2)   MME235-2014-15-(8)   MME235-2016-17-(7)
35        36  MME213-2017-18-(10)    ME221-2016-17-(5)   EEE267-2014-15-(2)
36        37    ME221-2016-17-(2)   MME235-2014-15-(1)   MME213-2015-16-(6)
37        38  MME213-2014-15-(10)    ME221-2017-18-(7)   MME235-2017-18-(2)
38        39    ME221-2018-19-(7)   MME235-2014-15-(2)   MME213-2015-16-(7)
39        40   MME235-2016-17-(5)   EEE267-2016-17-(2)   MME213-2017-18-(3)
40        41   MME235-2015-16-(3)   MME235-2015-16-(5)   MME213-2018-19-(3)
41        42   MME213-2014-15-(2)   MME213-2017-18-(7)    ME221-2014-15-(5)
42        43   EEE267-2017-18-(6)    ME221-2014-15-(2)    ME221-2015-16-(1)
43        44    ME221-2014-15-(8)   EEE267-2015-16-(4)  MME213-2017-18-(12)
44        45   MME213-2014-15-(1)   MME213-2015-16-(2)   MME213-2014-15-(6)
45        46   EEE267-2015-16-(1)    ME221-2016-17-(7)   MME235-2016-17-(4)
46        47    ME221-2016-17-(8)   MME235-2018-19-(1)   MME235-2018-19-(3)
47        48   MME235-2018-19-(7)   MME213-2017-18-(2)   EEE267-2014-15-(5)
48        49   MME235-2018-19-(5)  MME213-2014-15-(11)   MME235-2016-17-(1)
49        50   MME213-2018-19-(5)  MME213-2015-16-(10)   MME235-2017-18-(7)
50        51    ME221-2018-19-(6)   MME213-2014-15-(9)   MME235-2018-19-(8)
51        52   EEE267-2018-19-(7)   MME235-2015-16-(